# Adult Optimality analysis

In [1]:
import json, tensorflow as tf
from tensorflow.python.keras.backend import set_session

In [2]:
sess = tf.Session()

In [3]:
set_session(sess)

In [4]:
import numpy as np
np.random.seed(2)
chosen_success_idx = np.random.randint(0,90,(10))
seq_order_idx = np.random.permutation(936)
print(chosen_success_idx)
print(seq_order_idx[:10])

[40 15 72 22 43 82 75  7 34 49]
[903 512 200 143 806  15  65 199 687 356]


In [5]:
path = 'results_adult\\analysis\\adult\\result.%d.pruned.json'
success = []
failed = []

for i in range(0,100):
    a = json.load(open(path%i))
    if len(a['results'])==0:
        failed.append(i)
    else:
        success.append((i,a['results'][0]))
print(len(success), len(failed))

90 10


In [6]:
failed

[5, 16, 23, 56, 75, 80, 83, 92, 98, 99]

In [7]:
len(success)

90

In [8]:
type(chosen_success_idx)

numpy.ndarray

In [9]:
chosen_success = [success[i] for i in chosen_success_idx]

In [10]:
chosen_success

[(43,
  {'seq': ['AddEducation', 'ChangeWorkingHours', 'AddEducation'],
   'seq_str': 'AddEducation|ChangeWorkingHours|AddEducation',
   'p': [1.0239514112472534, 2.1067276000976562, 1.0239514112472534],
   'cost': 7.73970890045166,
   'l': 3,
   'grad_heur': 255,
   'vanilla': 41,
   'cost_heur': 38}),
 (17,
  {'seq': ['IncreaseCapitalGain', 'AddEducation', 'IncreaseCapitalGain'],
   'seq_str': 'IncreaseCapitalGain|AddEducation|IncreaseCapitalGain',
   'p': [1.2902129888534546, 1.1189450025558472, 1.2902129888534546],
   'cost': 4.932949542999268,
   'l': 3,
   'grad_heur': 465,
   'vanilla': 139,
   'cost_heur': 630}),
 (77,
  {'seq': ['AddEducation'],
   'seq_str': 'AddEducation',
   'p': [1.713195562362671],
   'cost': 4.342597961425781,
   'l': 1,
   'grad_heur': 0,
   'vanilla': 0,
   'cost_heur': 0}),
 (25,
  {'seq': ['AddEducation'],
   'seq_str': 'AddEducation',
   'p': [0.19495108723640442],
   'cost': 0.49887415766716003,
   'l': 1,
   'grad_heur': 0,
   'vanilla': 0,
   'co

In [11]:
import sys

In [3]:
from models.adult.model import Adult
from models.adult.actions import *
all_actions = {
    'AddEducation': AddEducation,
    'ChangeWorkingHours': ChangeWorkingHours,
    'ChangeCapitalLoss': ChangeCapitalLoss,
    'Enlist': Enlist,
    'IncreaseCapitalGain': IncreaseCapitalGain,
    'WaitYears': WaitYears
}
restore_path, data_path = 'model.h5', 'data.npy'

In [13]:
from local_pack.models.loader import load_model, load_env
env = load_env('adult', data_path, used_actions=None)
model = load_model('adult', restore_path)
data, actions, features, target_label = env

In [14]:
target_label

[0.0, 1.0]

In [15]:
from local_pack.heuristics.vanilla import VanillaHeuristics
Seq_Gen = VanillaHeuristics(actions=actions, max_length=4)
Seq_Gen.setup(None)
all_seq = Seq_Gen.sequences
print(len(all_seq))
print(sys.getsizeof(all_seq))

936
7984


In [16]:
model.model = tf.keras.models.load_model('local_pack\\models\\adult\\model.h5', custom_objects=dict(loss_fn=model.loss_fn))
model.model._make_predict_function()

In [17]:
def softmax(logits):
    return np.exp(logits)/np.sum(np.exp(logits))

In [18]:
def get_action_seq(seq):
    action_seq = [all_actions[a](features) for a in seq]
    i = 0
    num_params = sum([action.num_params for action in action_seq], 0)
    for j, action in enumerate(action_seq):
        action_seq[j] = action_seq[j].set_p_selector(i, num_params)
        i += action.num_params
    return action_seq

In [40]:
def apply_sequence(instance, sequence, params):
    path = [instance]
    cost = 0
    for i, action in enumerate(sequence):
        path.append(np.array(action.apply(instance=path[-1], p=params, use_tensor=False)))
        cost += action.get_cost(path[i].reshape((-1,)), path[i+1].reshape((-1,)), use_tensor=False)
    return path[-1], cost

In [20]:
for i, success_instance in chosen_success:
    seq = success_instance['seq']
    print(seq)
    action_seq = get_action_seq(seq)    
    instance = data.data[i].reshape((1,-1))
    final_instance, cost = apply_sequence(instance, action_seq, success_instance['p'])
    
    print(i)
    print('Original logits:', softmax(model.predict(instance)))
    print('Final logits:', softmax(model.predict(final_instance)))
    print('Logged cost:', success_instance['cost'])
    print('Calculated cost:', cost)
    
    print('\n\n\n')
    
    if i==5:
        break

['AddEducation', 'ChangeWorkingHours', 'AddEducation']
43
Original logits: [[0.69193536 0.30806464]]
Final logits: [[0.49941576 0.5005843 ]]
Logged cost: 7.73970890045166
Calculated cost: 7.755229654170012




['IncreaseCapitalGain', 'AddEducation', 'IncreaseCapitalGain']
17
Original logits: [[0.6002774 0.3997226]]
Final logits: [[0.49781796 0.502182  ]]
Logged cost: 4.932949542999268
Calculated cost: 4.779106742618377




['AddEducation']
77
Original logits: [[0.62774605 0.37225392]]
Final logits: [[0.49902445 0.5009755 ]]
Logged cost: 4.342597961425781
Calculated cost: 4.369259388397899




['AddEducation']
25
Original logits: [[0.51604545 0.48395455]]
Final logits: [[0.5000021  0.49999794]]
Logged cost: 0.49887415766716003
Calculated cost: 0.49719476684338915




['AddEducation']
46
Original logits: [[0.6015993 0.3984007]]
Final logits: [[0.49966934 0.5003306 ]]
Logged cost: 4.809963226318359
Calculated cost: 4.8311456944461035




['AddEducation']
89
Original logits: [[0.50278556 0

In [21]:
from tqdm import tqdm_notebook

In [68]:
def random_param_search(instance, seq):
    
    np.random.seed(5)
    
    p_min = -P_EXT
    p_max = P_EXT
    
    init_p = []
    for action in seq:
        init_p.extend(action.init_p)
        
    init_p = np.array(init_p)
    sh = init_p.shape[0]
    
    results = []
    
    min_cost = float("inf")
    opt_param = None
    best_final_inst = None
    
    for i in range(NUM_P_SEARCH*sh):
        rand_p = (np.random.rand(sh)-0.5)*(p_max-p_min) + (p_max+p_min)/2
        
        final_instance, cost = apply_sequence(instance, seq, rand_p)
        
        if np.argmax(target_label) == np.argmax(model.predict(np.reshape(final_instance,(1,-1)))):
            if cost<min_cost:
                opt_param = rand_p
                min_cost = cost
                best_final_inst = final_instance
        
    return opt_param, min_cost, best_final_inst

In [69]:
def search_successful_instance(instance, orig_seq, orig_param):
        
    orig_seq = get_action_seq(orig_seq)     
    
    final_instance, orig_cost = apply_sequence(instance, orig_seq, orig_param)
    
    search_param, search_cost, search_final_inst = random_param_search(instance, orig_seq)

    print('Original softmax:',softmax(model.predict(np.reshape(instance,(1,-1)))))
    print('Recalculated softmax:',softmax(model.predict(np.reshape(final_instance,(1,-1)))))
    if search_final_inst is None:
        print('No Solution found')
    else:
        print('Best search softmax:',softmax(model.predict(np.reshape(search_final_inst,(1,-1)))))

    
    return orig_cost, search_cost, search_param

In [70]:
def search_unsuccessful_instance(instance, seed):
    np.random.seed(seed)    
    
    min_cost = float('inf')
    best_params = None
    best_seq = None
    for i in tqdm_notebook(range(NUM_SEQ_SAMPLED)):
        test_seq = np.random.choice(all_seq)
        search_param, search_cost, search_final_inst = random_param_search(instance, test_seq)
        if search_final_inst is not None:
            print('Found!', test_seq)
            if search_cost<min_cost:
                best_params = search_param
                best_seq = test_seq
                min_cost = search_cost
    
    return best_seq, best_params, min_cost

In [71]:
len(all_seq)

936

In [72]:
P_EXT = 2
NUM_P_SEARCH = 3000
NUM_SEQ_SAMPLED = 1000

print(P_EXT, NUM_SEQ, NUM_P_SEARCH)
for i in failed:
    print(i)
    instance = data.data[i]
    best_seq, best_params, min_cost = search_unsuccessful_instance(instance, i)
    
    
    if best_seq is None:
        print('No solution found')
    else:
        print('Sequence:', best_seq, 'Parameters: ', best_params, 'Cost: ', min_cost)
    print('\n\n\n')

2 100 3000
5


KeyboardInterrupt: 